In [ ]:
%matplotlib inline
import librosa, librosa.display
import IPython.display as ipd
import vamp
import matplotlib.pyplot as plt
import numpy as np
import scipy.fftpack as fft

In [ ]:
import re
import collections

In [ ]:
import pprint
pp = pprint.PrettyPrinter(indent=4)
pprint = pp.pprint

In [ ]:
import vamp

In [ ]:
#path = '/home/sonja/Dropbox/RC/Music project/Autumn Leaves - Yenne Lee plays 2004 Pepe Romero Jr.-HxGT5z6d-GA.m4a'
#path = '/home/sonja/Dropbox/RC/Music project/Beatles - I wanna hold your hand - live.m4a'
#path = '/home/sonja/Dropbox/RC/Music project/Beatles - I wanna hold your hand -studio.m4a'
#path = '/home/sonja/Dropbox/RC/Music project/Dumb Ways to Die-IJNR2EpS0jw.m4a'
#path = '/home/sonja/Dropbox/RC/Music project/Queen - I Want to Break Free (Official Lyric Video)-WUOtCLOXgm8.m4a'
#path = '/home/sonja/Dropbox/RC/Music project/Selena Gomez, Marshmello - Wolves (Official Music Video)-cH4E_t3m3xM.m4a'
#path = '/home/sonja/Dropbox/RC/Music project/The Killers - Human-RIZdjT1472Y.m4a'
path = '/home/sonja/Dropbox/RC/Music project/THE MUFFIN SONG (asdfmovie feat. Schmoyoho)-LACbVhgtx9I.m4a'

In [ ]:
#HAS TO BE RUN FOR EVERY SONG
song = librosa.load(path)

In [ ]:
cow = librosa.load('/home/sonja/Dropbox/RC/Music project/Kuh muht-TdheW61w4Co.m4a')

In [ ]:
cow_chords = vamp.collect(cow[0], cow[1], "nnls-chroma:chordino")['list']

In [ ]:
#for cow
label = 'N'
ind = 0
while label == 'N':
    ind += 1
    label = cow_chords[ind]['label']
beginning_ind = ind
print(beginning_ind)
while label != 'N':
    ind += 1
    label = cow_chords[ind]['label']
end_ind = ind - 1
print(end_ind)

timestamps_cow = [float(cow_chords[beginning_ind]['timestamp']), float(cow_chords[2]['timestamp'])]
bounds_cow = [librosa.time_to_samples(t, cow[1]) for t in timestamps_cow]
print(bounds_cow)
bounds_cow[0]=bounds_cow[0]-25000
bounds_cow[1]=bounds_cow[0]+35000
cow_in_F = cow[0][bounds_cow[0]:bounds_cow[1]]
ipd.Audio(cow_in_F, rate=cow[1])

In [ ]:
#for cow
def last_not_0_before_par(par):
    ind = par
    while cow[0][ind] == 0:
        ind -= 1
    return ind

def first_0_before_par(par):
    ind = par
    while cow[0][ind] != 0:
        ind -= 1
    return ind

def sign_change(par):
    ind = par
    while cow[0][ind] * cow[0][par] > 0:
        ind -= 1
    return ind

In [ ]:
#for cow
last_int_sample = last_not_0_before_par(bounds_cow[1])
sample = last_int_sample
for _  in range(50):
    sample = sign_change(sample-1)
last_sample_extension = sample

for _  in range(1000):
    sample = sign_change(sample-1)
first_sample_extension = sample
    
#first_sample_extension = sample
print(last_int_sample)
print(first_sample_extension)

long_sample_cow = cow[0][bounds_cow[0]:bounds_cow[1]]
for _ in range(10):
    long_sample_cow = np.concatenate((long_sample_cow, cow[0][first_sample_extension:last_sample_extension]))
print(len(long_sample_cow))
ipd.Audio(long_sample_cow, rate=cow[1])

In [ ]:
#for cow
def long_enough_ignoring_frequencies(num_samples):
    output = cow[0][bounds_cow[0]:bounds_cow[1]]
    while len(output) < num_samples:
        output = np.concatenate((output, cow[0][first_sample_extension:last_sample_extension]))
    return output

In [ ]:
#for synth note
def note_by_num_samples(freq, num_samples, amp=1, rate=song[1]):
    data = np.array([np.sin(2*np.pi*freq*librosa.samples_to_time(sample, rate))*amp for sample in range(num_samples)], dtype=np.float32)
    return data

In [ ]:
encr_notes = {'C':0, 'C#':1, 'Db':1, 'D':2, 'D#':3, 'Eb':3, 'E':4, 'Fb':4, 'E#':5, 'F':5, 'F#':6, 'Gb':6, 'G':7, 'G#':8, 'Ab':8, 'A':9, 'A#':10, 'Bb':10, 'B':11, 'Cb':11, 'B#':0}
#pprint(encr_notes)

In [ ]:
notes_sharp_bass = [('C', 65.41), ('C#', 69.30), ('D', 73.42), ('D#', 77.78), ('E', 82.41), ('F', 87.31), ('F#', 92.50), ('G', 98.00), ('G#', 103.83), ('A', 110.00), ('A#', 116.54), ('B', 123.47)]
notes_sharp_medium_pitch = [('C', 130.81), ('C#',  138.59), ('D',  146.83), ('D#',  155.56), ('E',  164.81), ('F',  174.61), ('F#',  185.00), ('G',  196.00), ('G#',  207.65), ('A',  220.00), ('A#',  233.08), ('B',  246.94)]
#to do: make similar list for flat notes

In [ ]:
#HAS TO BE RUN FOR EVERY SONG
#find all chords and time stamps
chords = vamp.collect(song[0], song[1], "nnls-chroma:chordino")['list']
pprint(list(enumerate(chords)))

In [ ]:
#HAS TO BE RUN FOR EVERY SONG
#Find tonality:
last_chord_index = -1
while chords[last_chord_index]['label'] == 'N':
    last_chord_index -= 1
last_chord = chords[last_chord_index]['label']

list_of_chords = [chord['label'] for chord in chords]
numb_appearance_chord2 = collections.Counter(list_of_chords)
most_appearing_chord = numb_appearance_chord2.most_common(1)[0][0]


sign = ''
if len(most_appearing_chord) > 1:
    if most_appearing_chord[1] == '#' or most_appearing_chord[1] == 'b':
        sign = most_appearing_chord[1]

tonality = most_appearing_chord[0] + sign
print('Tonality:', tonality)
if last_chord != most_appearing_chord:
    print('Tonality might be wrong, sorry!')
    

In [ ]:
scale_degree_in_semitones = {1:0, 2:2, 3:4, 4:5, 5:7, 6:9, 7:11}

In [ ]:
#HAS TO BE RUN FOR EVERY SONG
#Compute the chord that corresponds to a certain scale degree:
scale_degree = 1
chord = notes_sharp_medium_pitch[(encr_notes[tonality] + scale_degree_in_semitones[scale_degree]) % 12]
print(chord)

In [ ]:
#HAS TO BE RUN FOR EVERY SONG
#Find the time stamps in between which the chord corresponding to the chosen scale degree appears in the song:
chord_pairs = list(zip(chords[:-1], chords[1:]))
timestamps_chord = [ (float(c[0]['timestamp']), float(c[1]['timestamp'])) for c in chord_pairs if re.match(re.escape(chord[0])+r'.*', c[0]['label'])]
bounds_chord = [librosa.time_to_samples(t, song[1]) for t in timestamps_chord]
lables_chord = [c[0]['label'] for c in chord_pairs if re.match(re.escape(chord[0])+r'.*', c[0]['label'])]

#chord_number = 2
#print(lables_chord[chord_number])
#ipd.Audio(song[0][bounds_chord[chord_number][0]:bounds_chord[chord_number][1]], rate=song[1])


In [ ]:
#Has TO BE TUN FOR EVERY SONG
#Compute highest amplitude in the song:
max_amplitude = max(song[0])
print(max_amplitude)

In [ ]:
#HAS TO BE RUN FOR EVERY SONG
#Generate the output (audio of song plus the base note of the chord corresponding to the scale degree chosen every time that chord appears):
add_to_song = np.zeros(len(song[0]))
for beginning, end in bounds_chord:
    if end > len(song[0]):
        end = len(song[0])
    add = note_by_num_samples(chord[1] , end-beginning, amp=(max_amplitude/2))
    add_to_song[beginning:end] = add
    
output = song[0] + add_to_song
ipd.Audio(output, rate=song[1])

In [ ]:
add_cow = np.zeros(len(song[0]))
max_num_samples = max([end-beginning for beginning, end in bounds_chord])
long_cow = long_enough_ignoring_frequencies(max_num_samples)
for beginning, end in bounds_chord:
    if end > len(song[0]):
        end = len(song[0])
    add = long_cow[:(end-beginning)]
    add_cow[beginning:end] = add

output = song[0] + add_cow
ipd.Audio(output, rate=song[1])

In [ ]:
#Generate the output (audio of song plus the base note of the chord corresponding to the scale degree chosen every time that chord appears):
add_to_song = np.zeros(len(song[0]))
for beginning, end in bounds_chord:
    if end > len(song[0]):
        end = len(song[0])
    add = note_by_num_samples(chord[1] , end-beginning, amp=(max_amplitude/3))
    add_to_song[beginning:end] = add
    
output = song[0] + add_to_song
ipd.Audio(output, rate=song[1])